In [1]:
import requests
from bs4 import BeautifulSoup
import json
import os
import datetime

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
}

base_url = "https://icook.tw"

response = requests.get(base_url, headers=headers)
response.encoding = "utf-8"  # 確保正確解碼

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    for title in soup.select("h2"):
        print(title.get_text(strip=True))
else:
    print("Failed to fetch:", response.status_code)

百萬網友都在使用的愛料理 APP!
我是發呆冠軍！韓爆紅「發呆大賽」6月台灣登場，同步加碼睡覺大賽、聞香大賽等你來挑戰
今晚吃什麼？就煮這3道！
端午節吃粽子！🚣‍♀️
大家一起做NEW
今日話題食譜
最新生活誌文章
食材處理方法
🥕 健身飲控
吃素 好好吃
新廚上菜
精選影音食譜
當季食材
最新食譜
iGood 好物推薦 🏠️
活動
品牌廚房
熱搜關鍵字
愛料理 App 看食譜做菜螢幕不變暗


In [4]:
search_recipe = "餅乾"
search_url = f"https://icook.tw/search/{search_recipe}"

response = requests.get(search_url, headers=headers)

root = BeautifulSoup(response.text, "html.parser")

links = set()

while True:
    recipe_items = root.find_all("li", class_="browse-recipe-item")

    for item in recipe_items:
        link = base_url + item.find("a", class_="browse-recipe-link")["href"]
        links.add(link)
        print(link)

    next_page = root.find("li", class_="pagination-tab page--next")
    if next_page:
        next_url = base_url + next_page.find("a")["href"]
        response = requests.get(next_url, headers=headers)
        root = BeautifulSoup(response.text, "html.parser")
    else:
        break

https://icook.tw/recipes/474792
https://icook.tw/recipes/471502
https://icook.tw/recipes/471112
https://icook.tw/recipes/473628
https://icook.tw/recipes/466969
https://icook.tw/recipes/474367
https://icook.tw/recipes/475359
https://icook.tw/recipes/475854
https://icook.tw/recipes/473930
https://icook.tw/recipes/475387
https://icook.tw/recipes/475303
https://icook.tw/recipes/476946
https://icook.tw/recipes/476164
https://icook.tw/recipes/475106
https://icook.tw/recipes/471223
https://icook.tw/recipes/474623
https://icook.tw/recipes/475599
https://icook.tw/recipes/471742
https://icook.tw/recipes/471742
https://icook.tw/recipes/473174
https://icook.tw/recipes/472797
https://icook.tw/recipes/473031
https://icook.tw/recipes/471763
https://icook.tw/recipes/473260
https://icook.tw/recipes/473392
https://icook.tw/recipes/470854
https://icook.tw/recipes/471862
https://icook.tw/recipes/471391
https://icook.tw/recipes/471336
https://icook.tw/recipes/471356
https://icook.tw/recipes/467679
https://

KeyboardInterrupt: 

In [5]:
def parse_chinese_number(num_str):
    num_str = num_str.replace(",", "").strip()
    if "萬" in num_str:
        try:
            # 去除萬字，轉換成float後乘以10000
            number = float(num_str.replace("萬", "").strip())
            return int(number * 10000)
        except ValueError:
            return None
    else:
        try:
            return int(num_str)
        except ValueError:
            return None

In [ ]:
recipes_dic = []

for link in links:
    print(f"Processing: {link}")
    response = requests.get(link, headers=headers)
    root = BeautifulSoup(response.text, "html.parser")
    if response.status_code != 200:
        print(f"無法取得食譜: {link} 狀態碼: {response.status_code}")
        continue
    title = root.find("title")

    if "愛料理上找不到您要的頁面喔～" in title.get_text(strip=True):
        print(f"無法取得食譜: {link} 找不到頁面")
        continue

    name = root.find("h1", id="recipe-name")
    is_vip = name.find("title")
    if is_vip and is_vip.get_text(strip=True) == "VIP 專屬":
        print(f"VIP 專屬食譜: {link}")
        continue

    username = root.find("div", class_="author-name")
    user_stats = root.find_all("span", class_="author-stat")
    for stat in user_stats:
        text = stat.get_text(strip=True)
        if "食譜" in text:
            try:
                user_recipe_str = stat.find("span", class_="stat-num").get_text(strip=True)
                user_recipe = parse_chinese_number(user_recipe_str)
                if user_recipe is None:
                    print(f"無法取得或轉換食譜數字: {user_recipe_str}")
            except AttributeError:
                user_recipe = None
                print("無法取得食譜數字: 無資料")
        elif "粉絲" in text:
            try:
                user_fans_str = stat.find("span", class_="stat-num").get_text(strip=True)
                user_fans = parse_chinese_number(user_fans_str)
                if user_fans is None:
                    print(f"無法取得或轉換粉絲數字: {user_fans_str}")
            except AttributeError:
                user_fans = None
                print("無法取得粉絲數字: 無資料")

    image_url = root.find("div", class_="ratio-container ratio-container-custom").find("img").get("src", None)
    description = root.find("section", class_="description")
    tags = root.find_all("li", class_="tag")
    tag_list = [tag.get_text(strip=True).replace("#", "") for tag in tags]
    left_block = root.find("span", class_="stat-left")
    if left_block and "說讚" in left_block.get_text(strip=True):
        like_text = left_block.get_text(strip=True).split("說讚")[0].strip()
        like_count = parse_chinese_number(like_text)
        if like_count is None:
            print(f"無法將 '{like_text}' 轉換為整數")
    right_block = root.find("span", class_="stat-right")
    if right_block:
        right_block_stat = right_block.find_all("a", class_="stat")
        for stat in right_block_stat:
            if "一起做" in stat.get_text(strip=True):
                together_text = stat.get_text(strip=True).split("一起做")[0].strip()
                together_count = parse_chinese_number(together_text)
                if together_count is None:
                    print(f"無法將 '{together_text}' 轉換為整數")
            elif "留言" in stat.get_text(strip=True):
                comment_text = stat.get_text(strip=True).split("留言")[0].strip()
                comment_count = parse_chinese_number(comment_text)
                if comment_count is None:
                    print(f"無法將 '{comment_text}' 轉換為整數")

    ingredient_list = []
    ingredients = root.find_all("li", class_="ingredient")
    for ingredient in ingredients:
        try:
            ingredient_name = ingredient.find("div", class_="ingredient-name")
            ingredient_amount = ingredient.find("div", class_="ingredient-unit")
            ingredient_group = ingredient.parent.parent.find("div", class_="group-name")
            ingredient_dic = {
                "name": ingredient_name.get_text(strip=True) if ingredient_name else None,
                "amount": ingredient_amount.get_text(strip=True) if ingredient_amount else None,
                "group": ingredient_group.get_text(strip=True) if ingredient_group else None
            }
            ingredient_list.append(ingredient_dic)
        except AttributeError:
            print(f"無法取得食材名稱或數量: {ingredient}")
            continue

    steps_dic = {}
    steps = root.find_all("li", class_="recipe-details-step-item")
    for count, step in enumerate(steps, start=1):
        try:
            step_text = step.find("p", class_="recipe-step-description-content").get_text(strip=True)
            steps_dic[count] = step_text
        except AttributeError:
            print(f"無法取得步驟描述: {step}")
            continue
        

    metas = root.find("div", class_="recipe-detail-metas")
    view_str = metas.find("div")
    try:
        if "瀏覽" in view_str.get_text(strip=True):
            view_str = view_str.get_text(strip=True).split("瀏覽")[0].strip()
            view_count = parse_chinese_number(view_str)
            if view_count is None:
                print(f"無法將 '{together_text}' 轉換為整數")
    except Exception as e:
        view_count = None
        print(f"錯誤: {e}")
    date = metas.find("time")['datetime']

    recipe = {
        "recipe_name": name.get_text(strip=True) if name else None,
        "recipe_link": link,
        "user_name": username.get_text(strip=True) if username else None,
        "user_recipe": user_recipe if 'user_recipe' in locals() else 0,
        "user_fans": user_fans if 'user_fans' in locals() else 0,
        "image_url": image_url if image_url else None,
        "description": description.get_text(strip=True) if description else None,
        "tags": tag_list if tag_list else [],
        "like_count": like_count if 'like_count' in locals() else 0,
        "together_count": together_count if 'together_count' in locals() else 0,
        "comment_count": comment_count if 'comment_count' in locals() else 0,
        "ingredients": ingredient_list if ingredient_list else [],
        "steps": steps_dic if steps_dic else {},
        "view_count": view_count if 'view_count' in locals() else 0,
        "date": date if date else None
    }
    recipes_dic.append(recipe)
    print(recipe)

Processing: https://icook.tw/recipes/235470
{'recipe_name': 'かにの雪Q餅', 'recipe_link': 'https://icook.tw/recipes/235470', 'user_name': 'かに小玉の幸福食堂', 'user_recipe': 279, 'user_fans': 862, 'description': '描述小時候的奇福餅乾與其他材料混合的口感，真是會讓人一口接一口，停不下來喔！配茶，配咖啡都很讚的。', 'tags': ['素食', '蔬食'], 'like_count': 213, 'together_count': 1, 'comment_count': 1, 'ingredients': [{'name': '棉花糖', 'amount': '200g', 'group': None}, {'name': '奶油', 'amount': '85g', 'group': None}, {'name': '蔓越莓乾', 'amount': '60g', 'group': None}, {'name': '奶粉', 'amount': '85g', 'group': None}, {'name': '奇福餅乾', 'amount': '350g', 'group': None}, {'name': '綜合堅果', 'amount': '210g', 'group': None}], 'steps': {1: '除了奶粉及奶油外，將材料敲或折或剪小塊', 2: '利用隔水加熱法~先融化奶油', 3: '奶油融化後，改放棉花糖，要不停的攪拌，讓它不黏鍋', 4: '棉花糖融化後，加入奶粉攪拌均勻', 5: '加入蔓越莓乾攪拌均勻', 6: '加入堅果碎拌勻', 7: '最後加入奇福餅乾碎拌勻，用煎鏟不好拌，可以直接用手的力道去拌勻', 8: '取一淺盤，鋪上烘焙紙 備用', 9: '拌勻的Q餅，趁餅乾糰還有熱度帶黏性時，倒入盤中舖平', 10: '用擀麵棍壓實，壓平', 11: '待冷卻後用刀裁切成適當大小', 12: '好吃的下午茶點心~雪Q餅', 13: '利用餅乾包裝袋個別包裝'}, 'view_count': 6293, 'date': '2018-01-10'}
P

In [ ]:
# current_time = datetime.datetime.now().strftime("%m%d_%H%M")

# output_path = f"output/data_{current_time}.json"

# with open(output_path, 'w', encoding='utf-8') as file:
#     json.dump(recipes_dic, file, ensure_ascii=False, indent=4)
    

# print(f"✅ JSON 檔案已儲存至: {output_path}")

In [ ]:
# current_time = datetime.datetime.now().strftime("%m%d_%H%M")

# output_path = f"output/data_{current_time}.jsonl"

# with open(output_path, 'w', encoding='utf-8') as f:
#     for item in recipes_dic:
#         json_line = json.dumps(item, ensure_ascii=False)  # 將 dict 轉成 JSON 字串
#         f.write(json_line + '\n')

# print(f"✅ JSONL 檔案已儲存至: {output_path}")

✅ JSON 檔案已儲存至: output/data_0510_2130.jsonl
